In [1]:
# You should be able to just run the whole script to align
# Just run all cells
#this
import os
import yaml
import time
import numpy as np
import pandas as pd
import shelve
import sys
import random
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import fsolve
sys.path.append('../')
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice
# from geecs_python_api.controls.experiment.experiment import Experiment



path config ..\..\..\..\user data\Configurations.INI
database name  loasis


In [5]:
#steup the geecs experiment
GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Bella")


In [11]:
"""This is the camera setup to get the centroid values and to average over some
number of images"""

camera = GeecsDevice('CAM-HPD-CCD')
camera.subscribe_var_values(['centroidx', 'centroidy'])
#camera.get('centroidx')
print(camera.state)

#inititalizing a empty array to store the centroid values and use it for running mean of 10
arr = []
#this variable defines the number of images to average over
num_ave = 10

for i in range(50):
    a = camera.state
    print(a)
    if len(a) > 3:
        #print(a['HPD_CCD_centroidx'],a['HPD_CCD_centroidy'])
        #get the x,y centroid values
        x = a[-2]
        y = a[-1]

        #append to the array till you have atleast num_ave values
        #once you have that, you can use it to get the running average
        if len(arr)<num_ave:
            arr.append([x,y])
        else:
            #This removes the first entry in the array
            #Then it appends the latest value at the end
            arr.remove(arr[0])
            arr.append([x,y])
            #print(i, np.mean(arr, axis = 0))
    time.sleep(0.2)
    
camera.close

{'fresh': True, 'shot number': None, 'GEECS device error': False}
{'fresh': True, 'shot number': None, 'GEECS device error': False}
{'fresh': True, 'shot number': None, 'GEECS device error': False}
{'fresh': True, 'shot number': None, 'GEECS device error': False}
{'fresh': True, 'shot number': None, 'GEECS device error': False}
{'fresh': True, 'shot number': None, 'GEECS device error': False}
{'fresh': True, 'shot number': 0, 'GEECS device error': False, 'last_message': 'CAM-HPD-CCD>>0>>Device Status nval,no scan nvar,\r\ndevice error nval, nvar,\r\ndevice preset nval,Amp2, MegaAttenuator nvar,\r\ncentroidx nval,1040.731323 nvar,\r\ncentroidy nval,1027.323364 nvar,\r\n', 'Device Status': 'no scan', '': '', 'device error': '', 'HPD_CCD_centroidx': 1040.731323, 'HPD_CCD_centroidy': 1027.323364}


KeyError: -2

In [21]:
list(a)[-2]

'HPD_CCD_centroidx'

In [58]:
"""This function returns the calibration factor for one of the 
axis at a time. It is the ratio of the voltage difference between 
positions and the amount by which the centroid moved.

Right now it requires that the lower and upper limit is defined
lower_lim = lower value of the mount at which the centroid dissapears from the camera
or the lower limit of the calibration you want to do

same for the upper limit

"""

def calibrate(camera, mount,axis ,lower_lim, upper_lim):
    if axis == "x":
        mount.set('X_Voltage', lower_lim)
        time.sleep(10)
        lower_centroid = centroid[0]
        mount.set('X_Voltage', upper_lim)
        time.sleep(10)
        upper_centroid = centroid[0]
        calib = (upper_lim-lower_lim)/(upper_centroid - lower_centroid)

    elif axis == "y":
        mount.set('Y_Voltage', lower_lim)
        time.sleep(10)
        lower_centroid = centroid[1]
        mount.set('Y_Voltage', upper_lim)
        time.sleep(10)
        upper_centroid = centroid[1]
        calib = (upper_lim-lower_lim)/(upper_centroid - lower_centroid)

    return(calib)
    
    

In [60]:
"""This function finds out the distance from the target and then calculates
the amount of moves we need to do"""

mount = GeecsDevice('')
mount.subscribe_var_values(['x_voltage', 'y_voltage'])
#camera.get('centroidx')
print(mount.state)

def steps_to_target(targetx, targety, mean):
    stepx = (targetx - mean[0])*calibx
    stepy = (targety - mean[1])*caliby
    result_arr = [stepx, stepy]
    return(result_arr)

def move_to_target(step_arr):
    mount.set('X_Voltage', step_arr[0])
    mount.set('Y_Voltage', step_arr[1])
    
    

{'fresh': True, 'shot number': None, 'GEECS device error': False}


In [3]:
import asyncio

def async_mean_centroid_acq(camera):
    
    